# Create data sets for LRG stellar mass completeness studies

In [ ]:
from pathlib import Path
from importlib import reload

from astropy.table import Table, vstack, hstack
import numpy as np

from desitarget.sv3.sv3_targetmask import desi_mask
from tqdm import tqdm
from joblib import Parallel, delayed
import utils
reload(utils)

In [ ]:
img_path = Path("/global/cfs/cdirs/cosmo/data/legacysurvey/dr9/")
mass_path = Path("/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass")
my_path = Path("/global/cscratch1/sd/bid13/LRG")
rongpu_path = Path("/global/cscratch1/sd/rongpu/target/catalogs/dr9.0/0.57.0")

In [ ]:
survey = "north" # choose from `north` or `south
selection = "IR" # choose from `OPT` or `IR`
n_workers=16

sweep_path = img_path / survey / "sweep" / "9.0"
mass_path = mass_path / survey
photo_z_path = img_path / survey / "sweep" / "9.0-photo-z"

## Make LRG catalogs from the sweep files

In [ ]:
load_path = my_path / f"LRG_{survey}.fits"
cat = []
sweep_files = list(sweep_path.glob("sweep*"))


# def process_sweep(sweep_file):
for sweep_file in tqdm(sweep_files):
    sweep = Table.read(sweep_file)
    quality_mask, sweep = utils.LRG_quality_cut(sweep)
    lrg_mask, sweep = utils.LRG_cut(sweep, survey=survey)

    keep_columns = ["OBJID", "gmag", "rmag", "zmag", "w1mag", "w2mag", "gfibermag", "rfibermag", 
                    "zfibermag"]
    sweep.keep_columns(keep_columns)
    photo_z = Table.read(photo_z_path / sweep_file.name.replace(".fits","-pz.fits"))
    sweep["zphot"] = photo_z[quality_mask][lrg_mask]["Z_PHOT_MEDIAN"]
    
    #This mass is based on photo-z
    logmass = np.load(mass_path / sweep_file.name.replace(".fits","_stellar_mass.npy") )
    sweep["logmass_photoz"] = logmass[quality_mask][lrg_mask]
    
    # return sweep
    cat.append(sweep)

# cat = Parallel(n_jobs=1)(delayed(process_sweep)(sweep_file) for sweep_file in tqdm(sweep_files))

print("\nSaving file...")
cat = vstack(cat, metadata_conflicts="silent")
cat.write(load_path, format="fits")
print(f"Catalog saved at {str(load_path)}")

## Make magnitude limited sample

In [ ]:
load_path = my_path / ("mag_lim_"+survey+".fits")
cat=[]
sweep_files = list(sweep_path.glob("sweep*"))

for ind, sweep_file in enumerate(sweep_files):
    sweep = Table.read(sweep_file)
    quality_mask, sweep = utils.LRG_quality_cut(sweep)
    
    sweep["gmag"] = utils.fluxToMag(sweep["FLUX_G"] / sweep["MW_TRANSMISSION_G"])
    sweep["rmag"] = utils.fluxToMag(sweep["FLUX_R"] / sweep["MW_TRANSMISSION_R"])
    sweep["zmag"] = utils.fluxToMag(sweep["FLUX_Z"] / sweep["MW_TRANSMISSION_Z"])
    sweep["w1mag"] = utils.fluxToMag(sweep["FLUX_W1"] / sweep["MW_TRANSMISSION_W1"])
    sweep["w2mag"] = utils.fluxToMag(sweep["FLUX_W2"] / sweep["MW_TRANSMISSION_W2"])
    sweep["zfibermag"] = utils.fluxToMag(sweep["FIBERFLUX_Z"] / sweep["MW_TRANSMISSION_Z"])
    
    stellar_mask = (
        sweep["zmag"] - sweep["w1mag"]
            > 0.8 * (sweep["rmag"] - sweep["zmag"]) - 0.6)
    if survey == "north":
        select_mask = (sweep["zfibermag"] < 21.61)
    if survey == "south":
        select_mask = (sweep["zfibermag"] < 21.6)
        
    sweep = sweep[select_mask & stellar_mask]
    
    keep_columns = ["OBJID", "gmag", "rmag","zmag","w1mag","w2mag", "zfibermag"]
#     keep_columns = ["OBJID", "EBV" ]
    sweep.keep_columns(keep_columns)
    
    photo_z = Table.read(photo_z_path / sweep_file.name.replace(".fits","-pz.fits"))
    sweep["zphot"] = photo_z[quality_mask][select_mask & stellar_mask]["Z_PHOT_MEDIAN"]
    sweep["zphot_std"] = photo_z[quality_mask][select_mask & stellar_mask]["Z_PHOT_STD"]

    logmass = np.load(mass_path / sweep_file.name.replace(".fits","_stellar_mass.npy") )
    sweep["logmass_photoz"] = logmass[quality_mask][select_mask & stellar_mask]

    cat.append(sweep)
    print(f"Processed file {ind+1} of {len(sweep_files)}", end="\r")
cat = vstack(cat, metadata_conflicts="silent")

In [ ]:
print("\nSaving file...")
cat.write(load_path, format="fits")
print(f"Catalog saved at {str(load_path)}")

# Make catalogs from official target lists generated by Rongpu

In [ ]:
basic_path = rongpu_path / ("dr9_sv3_lrg_"+ survey+ "_0.57.0_basic.fits")
photom_path = rongpu_path / ("dr9_sv3_lrg_"+ survey+ "_0.57.0_photom.fits")
vagc_path = rongpu_path / ("dr9_sv3_lrg_"+ survey+ "_0.57.0_more_1.fits")

In [ ]:
cat = Table.read(basic_path)
photo_cat = Table.read(photom_path)

# cat = hstack([cat, photo_cat])
vagc_cat = Table.read(vagc_path)
cat["gmag"] = utils.fluxToMag(photo_cat["FLUX_G"] / photo_cat["MW_TRANSMISSION_G"])
cat["rmag"] = utils.fluxToMag(photo_cat["FLUX_R"] / photo_cat["MW_TRANSMISSION_R"])
cat["zmag"] = utils.fluxToMag(photo_cat["FLUX_Z"] / photo_cat["MW_TRANSMISSION_Z"])
cat["w1mag"] = utils.fluxToMag(photo_cat["FLUX_W1"] / photo_cat["MW_TRANSMISSION_W1"])
cat["w2mag"] = utils.fluxToMag(photo_cat["FLUX_W2"] / photo_cat["MW_TRANSMISSION_W2"])
cat["zfibermag"] = utils.fluxToMag(photo_cat["FIBERFLUX_Z"] / photo_cat["MW_TRANSMISSION_Z"])
cat["zfibertotmag"] = utils.fluxToMag(photo_cat["FIBERTOTFLUX_Z"] )
cat["MORPHTYPE"] = photo_cat["MORPHTYPE"]

cat["zphot"] =  vagc_cat["Z_PHOT_MEDIAN"]
cat["logmass"] = vagc_cat["stellar_mass"]
cat["gaia_g"] = photo_cat["GAIA_PHOT_G_MEAN_MAG"]

load_path = my_path / ("LRG_SV3_" + survey +".fits")

cat.write(load_path, format="fits", overwrite=True)

# Make catalogs for BGS

In [ ]:
basic_path = rongpu_path / ("dr9_sv1_bgs_any_"+ survey+ "_0.49.0_basic.fits")
photom_path = rongpu_path / ("dr9_sv1_bgs_any_"+ survey+ "_0.49.0_photom.fits")
vagc_path = rongpu_path / ("dr9_sv1_bgs_any_"+ survey+ "_0.49.0_more_1.fits")

In [ ]:
cat = Table.read(basic_path)
photo_cat = Table.read(photom_path)

# cat = hstack([cat, photo_cat])
vagc_cat = Table.read(vagc_path)
cat["gmag"] = utils.fluxToMag(photo_cat["FLUX_G"] / photo_cat["MW_TRANSMISSION_G"])
cat["rmag"] = utils.fluxToMag(photo_cat["FLUX_R"] / photo_cat["MW_TRANSMISSION_R"])
cat["zmag"] = utils.fluxToMag(photo_cat["FLUX_Z"] / photo_cat["MW_TRANSMISSION_Z"])
cat["w1mag"] = utils.fluxToMag(photo_cat["FLUX_W1"] / photo_cat["MW_TRANSMISSION_W1"])
cat["w2mag"] = utils.fluxToMag(photo_cat["FLUX_W2"] / photo_cat["MW_TRANSMISSION_W2"])
cat["zfibermag"] = utils.fluxToMag(photo_cat["FIBERFLUX_Z"] / photo_cat["MW_TRANSMISSION_Z"])

cat["zphot"] =  vagc_cat["Z_PHOT_MEDIAN"]
cat["logmass"] = vagc_cat["stellar_mass"]

load_path = my_path / ("BGS_SV_" + survey +".fits")

cat.write(load_path, format="fits", overwrite=True)

In [ ]:
photo_cat = Table.read(rongpu_path / ("dr9_sv1_lrg_"+ survey+ "_0.49.0_more_1.fits"))

# Make LRG catalog from the official file generated by DESI_TARGET

In [ ]:
load_path = my_path / ("LRG_SV1.fits")
target_path = Path("/global/project/projectdirs/desi/target/catalogs/dr9/0.52.0/targets/sv1/resolve/dark")
cat = []
target_files = list(target_path.glob("sv1*"))
for ind, target_file in enumerate(target_files):
    target = Table.read(target_file)
    lrg_mask = (target["SV1_DESI_TARGET"] & desi_mask.mask("LRG")>0)
    
    target = target[lrg_mask]
    target["gmag"] = utils.fluxToMag(target["FLUX_G"] / target["MW_TRANSMISSION_G"])
    target["rmag"] = utils.fluxToMag(target["FLUX_R"] / target["MW_TRANSMISSION_R"])
    target["zmag"] = utils.fluxToMag(target["FLUX_Z"] / target["MW_TRANSMISSION_Z"])
    target["w1mag"] = utils.fluxToMag(target["FLUX_W1"] / target["MW_TRANSMISSION_W1"])
    target["zfibermag"] = utils.fluxToMag(target["FIBERFLUX_Z"] / target["MW_TRANSMISSION_Z"])
    target["zfibertotmag"] = utils.fluxToMag(target["FIBERTOTFLUX_Z"] )
    target["gaiagmag"] = target["GAIA_PHOT_G_MEAN_MAG"]
    keep_columns = ["TARGETID",  "FLUX_IVAR_W1", "FLUX_IVAR_W2", "RA", "DEC"]
                    # "gmag", "rmag", "zmag", "w1mag", "zfibermag", "zfibertotmag", "gaiagmag",
                    # "SV1_DESI_TARGET" ]
    target.keep_columns(keep_columns)
    

    cat.append(target)
    print(f"Processed file {ind+1} of {len(target_files)}", end="\r")

print("\nSaving file...")
cat = vstack(cat, metadata_conflicts="silent")
cat.write(load_path, format="fits")
print(f"Catalog saved at {str(load_path)}")